# Machine Learning for Level Truncation in String Field Theory

Consider the data of lumps in bosonic String Field Theory (SFT) and extrapolate level-$\infty$ predictions from finite level data.

In this notebook we use **stacking ensemble** learning to improve the predictions: we use two levels of learning algorithms splitting the training set into two subsets.

## Setup

First of all we print the characteristics of the current setup (OS, cores, etc.):

In [1]:
from mltools.libos import InfoOS

print('Current OS:                  {} (kernel release: {}, architecture: {})'.format(InfoOS().os, InfoOS().kernel, InfoOS().arch))
print('Number of available threads: {:d}'.format(InfoOS().threads))
print('Current CPU frequency:       {:.0f} MHz (max: {:.0f} MHz)'.format(InfoOS().freq, InfoOS().freqm))
print('Available RAM memory:        {:d} MB (tot: {:d} MB)'.format(InfoOS().vmav, InfoOS().vmtot))

Current OS:                  Linux (kernel release: 5.6.11-arch1-1, architecture: x86_64)
Number of available threads: 8
Current CPU frequency:       2991 MHz (max: 3800 MHz)
Available RAM memory:        7761 MB (tot: 15758 MB)


We establish early in the notebook the amount of cores we want to use in order to parallelize computations:

In [2]:
n_jobs = int(InfoOS().threads / 4)

We then check the installed versions of the packages we are going to use:

In [3]:
import sys

import matplotlib as mpl
import random     as rnd
import sklearn    as skl
import numpy      as np
import pandas     as pd

import warnings
warnings.simplefilter(action='ignore', category=UserWarning) # ignore user warnings: nothing that I can really do anything about it...


%matplotlib inline
mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# print the version of the modules
print('Python version: {:d}.{:d}'      .format(sys.version_info.major, sys.version_info.minor))
print('Matplot version: {}'            .format(mpl.__version__))
print('Numpy version: {}'              .format(np.__version__))
print('Pandas version: {}'             .format(pd.__version__))
print('Scikit-learn version: {}'       .format(skl.__version__))

# fix random_seed
RAND = 42
rnd.seed(RAND)
np.random.seed(RAND)

Python version: 3.7
Matplot version: 3.2.1
Numpy version: 1.18.4
Pandas version: 1.0.3
Scikit-learn version: 0.22.2.post1


## Session Preparation

In order to save the results of the analysis, we need to create the structure of directories in the current repository:

In [4]:
from os import path, makedirs

ROOT_DIR = '.' #-------------------------------------------------- root directory
IMG_DIR  = 'img' #------------------------------------------------ directory of images
MOD_DIR  = 'models' #--------------------------------------------- directory of saved models
LOG_DIR  = 'log' #------------------------------------------------ directory of logs
OUT_DIR  = 'output' #--------------------------------------------- directory for saved predictions, relevant output, etc.

DB_NAME = 'data_sft_analysis' #----------------------------------- name of the dataset
DB_FILE = DB_NAME + '.h5' #--------------------------------------- full name with extension
DB_PATH = path.join(ROOT_DIR, DB_FILE) #-------------------------- full path of the dataset

# define full paths
IMG_PATH = path.join(ROOT_DIR, IMG_DIR)
MOD_PATH = path.join(ROOT_DIR, MOD_DIR)
LOG_PATH = path.join(ROOT_DIR, LOG_DIR)
OUT_PATH = path.join(ROOT_DIR, OUT_DIR)

# create directories if non existent
if not path.isdir(IMG_PATH):
    makedirs(IMG_PATH, exist_ok=True)
if not path.isdir(MOD_PATH):
    makedirs(MOD_PATH, exist_ok=True)
if not path.isdir(LOG_PATH):
    makedirs(LOG_PATH, exist_ok=True)
if not path.isdir(OUT_PATH):
    makedirs(OUT_PATH, exist_ok=True)

Then create a logging session to store debug information:

In [5]:
import logging

from mltools.liblog import create_logfile

path_to_log = path.join(LOG_PATH,
                        DB_NAME + '_stacking.log'
                       ) #----------------------------------------------------- log path
log = create_logfile(path_to_log,
                     name=DB_NAME,
                     level=logging.DEBUG
                    ) #-------------------------------------------------------- create log file and session

# these lines provide the same setup also for the Jupyter logging
logger = logging.getLogger() #------------------------------------------------- get the current logging session

fmt = logging.Formatter('%(asctime)s: %(levelname)s ==> %(message)s') #-------- customise the formatting options

handler = logging.StreamHandler() #-------------------------------------------- handle the stream to the default (stderr)
handler.setLevel(logging.DEBUG) #---------------------------------------------- print everything
handler.setFormatter(fmt) #---------------------------------------------------- set the formatting options

logger.handlers = [handler] #-------------------------------------------------- override the default stream

# we are ready to go!
log.info('New logging session started. Log is at {}.'.format(path_to_log))

Rotating existing logs...


2020-05-11 15:41:22,316: INFO ==> New logging session started. Log is at ./log/data_sft_analysis_stacking.log.


## Import the Database

We then import the database from its JSON format and begin to analyse it:

In [6]:
if path.isfile(DB_PATH):
    df = pd.read_hdf(DB_PATH)
    log.debug('Database loaded.')
else:
    print('Database is not in the file tree!')
    log.error('Cannot find database!')

2020-05-11 15:41:22,807: DEBUG ==> Database loaded.


We then show the `dtypes` of each column to get an idea of the data structure:

In [7]:
df.dtypes

system      int64
type        int64
weight    float64
2         float64
3         float64
4         float64
5         float64
6         float64
7         float64
8         float64
9         float64
10        float64
11        float64
12        float64
13        float64
14        float64
15        float64
16        float64
17        float64
18        float64
exp         int64
dtype: object

In the dataset we have the predictions of the position of lumps in bosonic SFT for finite levels in the numbered columns and the extrapolation for level-$\infty$ in the column _exp_. We want to use known data (including the _weight_ and the _type_ of the input data) to predict the _exp_ labels (_init_ in principle can be left out).

In [8]:
df.head(3)

,system,type,weight,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,exp
0,40,2,0.0,1.169090,1.103953,1.049652,1.042422,1.025771,1.023925,1.016453,...,1.011730,1.011495,1.008954,1.008871,1.007160,1.007141,1.005919,1.005930,1.005018,1
1,26,2,0.0,1.133441,1.073850,1.041536,1.033556,1.022857,1.020416,1.015351,...,1.011415,1.010882,1.009026,1.008720,1.007436,1.007245,1.006306,1.006180,1.005465,1
2,27,2,0.0,1.140716,1.076342,1.042333,1.034286,1.023112,1.020708,1.015447,...,1.011446,1.010940,1.009026,1.008740,1.007419,1.007244,1.006281,1.006167,1.005435,1


## Training Set Prearation and Validation Strategy

In this case the division into training and test set can be a bit tricky. In fact we want to separate the samples according to their reference `system` in order to keep entries coming from the same "family" together. We will then keep 20% of the `system` values as test set, with no regards to the effective number of samples in the set.

In [9]:
from sklearn.model_selection import train_test_split

system_train, system_test = train_test_split(df['system'].unique(), test_size=0.20, shuffle=True, random_state=RAND)

We then split the training set into two subsets. In the first level set we keep around 70% of the remaining samples, while we keep only 30% of them in the second level.

In [10]:
system_train_lv1, system_train_lv2 = train_test_split(system_train, test_size=0.3, shuffle=True, random_state=RAND)

The samples are therefore inserted in their sets as:

- 20% in the test set,
- 56% in the first level set (70% of the total training set),
- 24% in the second level set (30% of the total training set).

We then form the training and test sets using `system_train_lv1`, `system_train_lv2` and `system_test` as index:

In [11]:
df_train_lv1 = df.loc[df['system'].isin(system_train_lv1)]
df_train_lv2 = df.loc[df['system'].isin(system_train_lv2)]
log.debug('Length of the lv1 training dataset: {:d} samples.'.format(df_train_lv1.shape[0]))
log.debug('Length of the lv2 training dataset: {:d} samples.'.format(df_train_lv2.shape[0]))

df_test  = df.loc[df['system'].isin(system_test)]
log.debug('Length of the test dataset: {:d} samples.'.format(df_test.shape[0]))

print('Training data (lv1): {:d}% of the dataset.'.format(int(100 * df_train_lv1.shape[0] / df.shape[0])))
print('Training data (lv2): {:d}% of the dataset.'.format(int(100 * df_train_lv2.shape[0] / df.shape[0])))
print('Test data:           {:d}% of the dataset.'.format(int(100 * df_test.shape[0] / df.shape[0])))

2020-05-11 15:41:23,020: DEBUG ==> Length of the lv1 training dataset: 398 samples.
2020-05-11 15:41:23,026: DEBUG ==> Length of the lv2 training dataset: 171 samples.
2020-05-11 15:41:23,030: DEBUG ==> Length of the test dataset: 149 samples.


Training data (lv1): 55% of the dataset.
Training data (lv2): 23% of the dataset.
Test data:           20% of the dataset.


Even though the number of samples in each split has been reduced with respect to the previous analysis, we keep using **cross-validation** to score the algorithms. However we have to reduce the number of splits to produce a meaningful prediction. We will therefore divide the original training set as follows:

In [12]:
from sklearn.model_selection import KFold

cv_lv1 = KFold(n_splits=3, shuffle=True, random_state=RAND)
cv_lv2 = KFold(n_splits=2, shuffle=True, random_state=RAND)

## Machine Learning Analysis

We now move to the ML analysis of the dataset for the **first level** learning. We consider:

- [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) to have a solid baseline for comparison,
- [SVR (Gaussian kernel)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) to hopefully find better results,
- [Histogram Gradient Boosting Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html) to improve the predictive abilities of a single decision tree through successive improvement.

For the hyperparameter optimization we use a [**Bayesan** approach](https://en.wikipedia.org/wiki/Bayesian_optimization) in the [_Scikit-optimize_](https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html) library: this will provide a better approach to the minimization of the cost functions during cross-validation with respect to a randomized search (apart from the linear regression where we will test all possible values of the hyperparameters). In order to print the output of the parameters dictionaries we implement a function to pretty print them:

In [13]:
def pretty(dct, indent=True):
    '''
    Pretty print the dictionary of best parameters.
    
    Required argument:
        dct:    the dictionary to pretty print.
        
    Optional argument:
        indent: whether to indent the printed output.
    '''
    
    for key, value in dct.items():
        if indent:
            print('    {} = {}'.format(key, value))
        else:
            print('{} = {}'.format(key, value))

### Features and Labels Extraction

We then extract the training features and the labels. If needed we can implement the scaling of the features.

In [14]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# drop the `system` column
df_train_lv1 = df_train_lv1.drop(columns='system')
df_train_lv2 = df_train_lv2.drop(columns='system')
df_test      = df_test.drop(columns='system')

# features
features_train_lv1 = df_train_lv1.drop(columns='exp')
features_train_lv2 = df_train_lv2.drop(columns='exp')
features_test      = df_test.drop(columns='exp')

# labels
labels_train_lv1 = df_train_lv1['exp']
labels_train_lv2 = df_train_lv2['exp']
labels_test      = df_test['exp']

### [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

We first consider a linear regression algorithm. In this case the number of hyperparameters to be tuned is small and we can use a **grid search** to try out every combination:

- `fit_intercept` $\in \lbrace 0, 1 \rbrace$,
- `normalize` $\in \lbrace 0, 1 \rbrace$.

In [15]:
from sklearn.linear_model    import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics         import mean_squared_error
from mltools.libscore        import ViewCV
from mltools.libplot         import Plot

import joblib

# parameter dictionary
search_params  = {'fit_intercept': [0, 1],
                  'normalize':     [0, 1]
                 } #----------------------------------------------------------------------------- define hyperparameter grid

# optimization search
log.info('LINEAR REGRESSION')
lin_reg = GridSearchCV(estimator=LinearRegression(),
                       param_grid=search_params,
                       scoring='neg_mean_squared_error',
                       n_jobs=-1,
                       refit=True,
                       cv=cv_lv1
                      ) #----------------------------------------------------------------------- define the optimization estimator

# fit the estimator
log.info('Fitting the estimator...')
lin_reg.fit(features_train_lv1, labels_train_lv1)

# evaluate the estimator
log.info('Evaluating the estimator...')
cv_results = ViewCV(lin_reg)

print('\nBest parameters:\n')
pretty(cv_results.best_parameters)

print('\nRMSE on the validation set: {:.3f} ± {:.3f}'.format(np.sqrt(-cv_results.test_mean()),
                                                             np.sqrt(cv_results.test_std())
                                                            )
     )

2020-05-11 15:41:23,243: INFO ==> LINEAR REGRESSION
2020-05-11 15:41:23,245: INFO ==> Fitting the estimator...
2020-05-11 15:41:26,068: INFO ==> Evaluating the estimator...



Best parameters:

    fit_intercept = 1
    normalize = 0

RMSE on the validation set: 0.480 ± 0.282


### [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) (Gaussian Kernel)

We now implement a kernel function in the SVM. In particular we use the Gaussian kernel (_rbf_) and try to approximate the level-$\infty$ predictions using these hyperparameters:

- `gamma` $\in \left[ 10^{-1}, 10^1 \right]$,
- `C` $\in \left[ 10^{-1}, 5 \times 10^2 \right]$,
- `epsilon` $\in \left[ 10^{-4}, 10^{-1} \right]$,
- `shrinking` $\in \lbrace 0, 1 \rbrace$.

In [16]:
from sklearn.svm      import SVR
from skopt            import BayesSearchCV
from skopt.space      import Real, Integer, Categorical
from sklearn.metrics  import mean_squared_error
from mltools.libscore import ViewCV
from mltools.libplot  import Plot

import joblib

# parameter dictionary
search_params  = {'epsilon':   Real(1.0e-4, 1.0e-1, prior='log-uniform'),
                  'C':         Real(1.0e-1, 5.0e2,  prior='log-uniform'),
                  'gamma':     Real(1.0e-1, 1.0e1,  prior='log-uniform'),
                  'shrinking': Integer(0, 1)
                 } #----------------------------------------------------------------------------------- define hyperparameter grid

# optimization search
log.info('SVR')
svr = BayesSearchCV(SVR(kernel='rbf'),
                    search_spaces=search_params,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1,
                    refit=True,
                    n_iter=50,
                    random_state=RAND,
                    cv=cv_lv1
                   ) #-------------------------------------------------------------------------------- define the optimization estimator

# fit the estimator
log.info('Fitting the estimator...')
svr.fit(features_train_lv1, labels_train_lv1)

# evaluate the estimator
log.info('Evaluating the estimator...')
cv_results = ViewCV(svr)

print('\nBest parameters:\n')
pretty(cv_results.best_parameters)

print('\nRMSE on the validation set: {:.3f} ± {:.3f}'.format(np.sqrt(-cv_results.test_mean()),
                                                             np.sqrt(cv_results.test_std())
                                                            )
     )

2020-05-11 15:41:26,344: INFO ==> SVR
2020-05-11 15:41:26,350: INFO ==> Fitting the estimator...
2020-05-11 15:44:39,686: INFO ==> Evaluating the estimator...



Best parameters:

    C = 22.524558668596875
    epsilon = 0.0001
    gamma = 0.1
    shrinking = 1

RMSE on the validation set: 0.443 ± 0.182


### [Histogram Based Gradient Boosting Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html)

We consider the new (still _experimental_) version of _Scikit_ **gradient boosting** algorithm, which is **histogram bases**. This should automatically handle _NaN_ gradients and be quite faster than the usual implementation. We will consider the followin hyperparameters:

- `loss` $\in \lbrace least\_squares, least\_absolute\_deviation \rbrace$,
- `learning_rate` $\in \left[ 10^{-4}, 10^{-1} \right]$,
- `max_iter` $\in \left[ 10, 300 \right]$,
- `max_depth` $\in \left[ 2, 100 \right]$,
- `min_samples_leaf` $\in \left[ 10, 100 \right]$,
- `l2_regularization` $\in \left[ 10^{-6}, 10^2 \right]$,
- `max_leaf_nodes` $\in \left[ 2, 50 \right]$.

In [17]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble     import HistGradientBoostingRegressor
from skopt                import BayesSearchCV
from skopt.space          import Real, Integer, Categorical
from sklearn.metrics      import mean_squared_error
from mltools.libscore     import ViewCV
from mltools.libplot      import Plot

import joblib

# parameter dictionary
search_params = {'max_iter':          Integer(1.0e1, 3.0e2, prior='log-uniform'),
                 'max_depth':         Integer(2, 100,  prior='uniform'),
                 'max_leaf_nodes':    Integer(2, 50,   prior='uniform'),
                 'min_samples_leaf':  Integer(10, 100, prior='uniform'),
                 'learning_rate':     Real(1.0e-4, 1.0e-1, prior='log-uniform'),
                 'l2_regularization': Real(1.0e-6, 1.0e2,  prior='log-uniform'),
                 'loss':              Categorical(['least_squares', 'least_absolute_deviation'])
                } #------------------------------------------------------------------------------------- define the hyperparameter grid

# optimization search
log.info('BOOSTED TREES')
grd_boost = BayesSearchCV(HistGradientBoostingRegressor(scoring='loss', validation_fraction=None, n_iter_no_change=10, random_state=RAND),
                          search_spaces=search_params,
                          scoring='neg_mean_squared_error',
                          n_jobs=-1,
                          refit=True,
                          n_iter=25,
                          random_state=RAND,
                          cv=cv_lv1
                         ) #--------------------------------------------------------------------------- define the optimization estimator

# fit the estimator
log.info('Fitting the estimator...')
grd_boost.fit(features_train_lv1, labels_train_lv1)

# evaluate the estimator
log.info('Evaluating the estimator...')
cv_results = ViewCV(grd_boost)

print('\nBest parameters:\n')
pretty(cv_results.best_parameters)

print('\nRMSE on the validation set: {:.3f} ± {:.3f}'.format(np.sqrt(-cv_results.test_mean()),
                                                             np.sqrt(cv_results.test_std())
                                                            )
     )

2020-05-11 15:44:39,735: INFO ==> BOOSTED TREES
2020-05-11 15:44:39,736: INFO ==> Fitting the estimator...
2020-05-11 15:54:16,696: INFO ==> Evaluating the estimator...



Best parameters:

    l2_regularization = 0.01753175923543476
    learning_rate = 0.1
    loss = least_squares
    max_depth = 100
    max_iter = 76
    max_leaf_nodes = 22
    min_samples_leaf = 10

RMSE on the validation set: 0.158 ± 0.110


## Predictions Stacking

We then stack the predictions for the second level training set and the test set.

In [18]:
# stack the predictions for the 2nd level set
predictions_lv2 = np.c_[lin_reg.best_estimator_.predict(features_train_lv2),
                        svr.best_estimator_.predict(features_train_lv2),
                        grd_boost.best_estimator_.predict(features_train_lv2)
                       ]

# stack the predictions for the test set
predictions_test = np.c_[lin_reg.best_estimator_.predict(features_test),
                         svr.best_estimator_.predict(features_test),
                         grd_boost.best_estimator_.predict(features_test)
                        ]

## Meta Learning

We then train the meta learner. We compare the performance of three different algorithms:

- Linear Regression,
- Boosted Trees,
- Neural Networks.

### Linear Regression

In [19]:
from sklearn.linear_model    import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics         import mean_squared_error
from mltools.libscore        import ViewCV
from mltools.libplot         import Plot

import joblib

# parameter dictionary
search_params  = {'fit_intercept': [0, 1],
                  'normalize':     [0, 1]
                 } #----------------------------------------------------------------------------- define hyperparameter grid

# optimization search
log.info('LINEAR REGRESSION')
lin_reg_meta = GridSearchCV(estimator=LinearRegression(),
                            param_grid=search_params,
                            scoring='neg_mean_squared_error',
                            n_jobs=-1,
                            refit=True,
                            cv=cv_lv2
                           ) #----------------------------------------------------------------------- define the optimization estimator

# fit the estimator
log.info('Fitting the estimator...')
lin_reg_meta.fit(predictions_lv2, labels_train_lv2)

# evaluate the estimator
log.info('Evaluating the estimator...')
cv_results = ViewCV(lin_reg_meta)

print('\nBest parameters:\n')
pretty(cv_results.best_parameters)

print('\nRMSE on the validation set: {:.3f} ± {:.3f}'.format(np.sqrt(-cv_results.test_mean()),
                                                             np.sqrt(cv_results.test_std())
                                                            )
     )

print('\nRMSE of the test predictions: {:.3f}'.format(np.sqrt(mean_squared_error(y_true=labels_test, y_pred=lin_reg_meta.predict(predictions_test)))))

2020-05-11 15:54:23,052: INFO ==> LINEAR REGRESSION
2020-05-11 15:54:23,053: INFO ==> Fitting the estimator...
2020-05-11 15:54:25,383: INFO ==> Evaluating the estimator...



Best parameters:

    fit_intercept = 0
    normalize = 0

RMSE on the validation set: 0.161 ± 0.010

RMSE of the test predictions: 0.087


## Gradient Boosting

In [20]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble     import HistGradientBoostingRegressor
from skopt                import BayesSearchCV
from skopt.space          import Real, Integer, Categorical
from sklearn.metrics      import mean_squared_error
from mltools.libscore     import ViewCV
from mltools.libplot      import Plot

import joblib

# parameter dictionary
search_params = {'max_iter':          Integer(1.0e1, 3.0e2, prior='log-uniform'),
                 'max_depth':         Integer(2, 100,  prior='uniform'),
                 'max_leaf_nodes':    Integer(2, 50,   prior='uniform'),
                 'min_samples_leaf':  Integer(10, 100, prior='uniform'),
                 'learning_rate':     Real(1.0e-4, 1.0e-1, prior='log-uniform'),
                 'l2_regularization': Real(1.0e-6, 1.0e2,  prior='log-uniform'),
                 'loss':              Categorical(['least_squares', 'least_absolute_deviation'])
                } #------------------------------------------------------------------------------------- define the hyperparameter grid

# optimization search
log.info('BOOSTED TREES')
grd_boost_meta = BayesSearchCV(HistGradientBoostingRegressor(scoring='loss', validation_fraction=None, n_iter_no_change=10, random_state=RAND),
                               search_spaces=search_params,
                               scoring='neg_mean_squared_error',
                               n_jobs=-1,
                               refit=True,
                               n_iter=25,
                               random_state=RAND,
                               cv=cv_lv2
                              ) #--------------------------------------------------------------------------- define the optimization estimator

# fit the estimator
log.info('Fitting the estimator...')
grd_boost_meta.fit(predictions_lv2, labels_train_lv2)

# evaluate the estimator
log.info('Evaluating the estimator...')
cv_results = ViewCV(grd_boost_meta)

print('\nBest parameters:\n')
pretty(cv_results.best_parameters)

print('\nRMSE on the validation set: {:.3f} ± {:.3f}'.format(np.sqrt(-cv_results.test_mean()),
                                                             np.sqrt(cv_results.test_std())
                                                            )
     )

print('\nRMSE of the test predictions: {:.3f}'.format(np.sqrt(mean_squared_error(y_true=labels_test, y_pred=grd_boost_meta.predict(predictions_test)))))

2020-05-11 15:54:25,452: INFO ==> BOOSTED TREES
2020-05-11 15:54:25,466: INFO ==> Fitting the estimator...
2020-05-11 16:10:03,506: INFO ==> Evaluating the estimator...



Best parameters:

    l2_regularization = 0.010732312983479157
    learning_rate = 0.1
    loss = least_squares
    max_depth = 33
    max_iter = 228
    max_leaf_nodes = 30
    min_samples_leaf = 10

RMSE on the validation set: 0.156 ± 0.089

RMSE of the test predictions: 0.033


## Neural Network

In [23]:
import tensorflow as tf
tf.random.set_seed(RAND)

from tensorflow.keras            import Sequential
from tensorflow.keras.layers     import Dense, BatchNormalization, Dropout, LeakyReLU, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks  import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils      import model_to_dot

# create validation set
predictions_lv2_train, predictions_lv2_val, \
labels_train_lv2_train, labels_train_lv2_val = train_test_split(predictions_lv2, labels_train_lv2,
                                                                test_size=0.5,
                                                                shuffle=True,
                                                                random_state=RAND)

# create the model
tf.keras.backend.clear_session()

model = Sequential(name='lv2_training')

model.add(InputLayer(input_shape=(predictions_lv2_train.shape[1],), name='lv1_predictions'))

model.add(Dense(3, name='dense_layer_1'))
model.add(LeakyReLU(alpha=0.1, name='dense_layer_1_activation'))

model.add(Dense(1, name='exp'))

# compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_squared_error'])
model.summary()
model_dot = model_to_dot(model=model, show_shapes=True)
model_dot.write_pdf(path.join(IMG_PATH, 'neural_network_stacking.pdf'))

# fit the model
callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=1000, verbose=0),
             ReduceLROnPlateau(monitor='val_mean_squared_error', factor=0.33, patience=750, verbose=0),
             ModelCheckpoint(filepath=path.join(MOD_PATH, 'neural_network_stacking.h5'), monitor='val_mean_squared_error', save_best_only=True, verbose=0)
            ]
model_history = model.fit(x=predictions_lv2_train,
                          y=labels_train_lv2_train.values,
                          batch_size=predictions_lv2_train.shape[0],
                          epochs=10000,
                          callbacks=callbacks,
                          validation_data=(predictions_lv2_val, labels_train_lv2_val.values),
                          verbose=0
                         )

# evaluate the model
val_evaluation  = model.evaluate(x=predictions_lv2_val, y=labels_train_lv2_val.values, verbose=0)
print('\nRMSE of the validation set predictions: {:.3f}'.format(np.sqrt(val_evaluation[1])))
test_evaluation = model.evaluate(x=predictions_test, y=labels_test.values, verbose=0)
print('RMSE of the test set predictions:       {:.3f}'.format(np.sqrt(test_evaluation[1])))

Model: "lv2_training"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer_1 (Dense)        (None, 3)                 12        
_________________________________________________________________
dense_layer_1_activation (Le (None, 3)                 0         
_________________________________________________________________
exp (Dense)                  (None, 1)                 4         
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________

RMSE of the validation set predictions: 0.142
RMSE of the test set predictions:       0.106


## Discussion and Data Saving

After training the algorithms, it seems that simply using a linear regression model as meta learner improves the predictive abilities of the model. The neural networks improves the model with very few parameters. Finally the gradient boosting seems to provide the best choice.

In [24]:
import json

predictions = {'exp_true':       labels_test.tolist(),
               'lin_reg':        lin_reg_meta.best_estimator_.predict(predictions_test).tolist(),
               'grd_boost':      grd_boost_meta.best_estimator_.predict(predictions_test).tolist(),
               'neural_network': model.predict(predictions_test).tolist()
              }

with open(path.join(OUT_PATH, 'stacking_results.json'), 'w') as f:
    json.dump(predictions, f)